# Naive Bayes Classifier

## 1. Naive Bayes algorithm

This ipython notebook is used to implement naive bayes algorithms, which is easy to understand and for some problem it has a fair good accuracy. 

**The Question we are going to answer **:

This is the simple example I get from wikipedia, which is used to understand how Naive Bayes algorithm works in this case:

### Problem: classify whether a given person is a male or a female based on the measured features. The features include height, weight, and foot size.

In [2]:
## Import the packages

import numpy as np
import matplotlib as plt
import sklearn as sk

%matplotlib notebook

In [4]:
## dataset columns: [height(feet), weight(lbs), foot size(inches), gender(male:1, female:0)]


dataset = np.array([[6,180,12,1],[5.92,190,11,1],[5.58,170,12,1],[5.92,165,10,1],[5,100,6,0],[5.5,150,8,0],[5.42,130,7,0],[5.75,150,9,0]],dtype=float)

In [5]:
dataset

array([[   6.  ,  180.  ,   12.  ,    1.  ],
       [   5.92,  190.  ,   11.  ,    1.  ],
       [   5.58,  170.  ,   12.  ,    1.  ],
       [   5.92,  165.  ,   10.  ,    1.  ],
       [   5.  ,  100.  ,    6.  ,    0.  ],
       [   5.5 ,  150.  ,    8.  ,    0.  ],
       [   5.42,  130.  ,    7.  ,    0.  ],
       [   5.75,  150.  ,    9.  ,    0.  ]])

In [22]:
train_x = dataset[:,:-1]

target = dataset[:,-1].reshape((1,dataset.shape[0]))

In [26]:
np.unique(target[0])

array([ 0.,  1.])

## Implement our Bayes algorithm

In [18]:
## define Gaussian distribution function

def Gauss(x, mean, var):
    """
    This is the function to calculate the probability distribution under the assumption that the data behaves as normla distribution
    """
    coef = 1/np.sqrt(2*np.pi*var)
    expo = np.exp((x-mean)**2/(2*var))
    return coef*expo


def Mean_array(train_X,target):
    """
    This function calculate the mean value for each feature (colume of dataset),
    this function is actually build the training model
    
    input: training array with size m*n
           m: the number of observations
           n: the number of features
    
    return: numpy array of mean values. with size 1*n
            n: the number of features
    """
    
    mean_array = np.array([[train_X[:,i].mean() for i in range(train_X.shape[1])]])
    
    assert mean_array.shape==(1,train_X.shape[1])
    
    return mean_array


def Var_array(train_X,target):
    """
    This function calculate the variance value for each feature (colume of dataset),
    this function is actually build the training model
    
    input: training array with size m*n
           m: the number of observations
           n: the number of features
    
    return: numpy array of variance values. with size 1*n
            n: the number of features
    """
    
    var_array = np.array([[train_X[:,i].var() for i in range(train_X.shape[1])]])
    
    assert var_array.shape==(1,train_X.shape[1])
    
    return var_array




In [19]:
Var_array(dataset)

array([[  9.74484375e-02,   7.33984375e+02,   4.48437500e+00,
          2.50000000e-01]])